# MEMO Assignment
Andrea De Carlo and Pooya Torabi

In [ ]:
# !pip install transformers
# todo need to move installation to requirements.txt

In [14]:
# Model
# clip-ViT-B-16, https://huggingface.co/openai/clip-vit-base-patch16

import requests
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

pretrained_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

In [15]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)
outputs = pretrained_model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

print(probs)

tensor([[9.9916e-01, 8.4369e-04]], grad_fn=<SoftmaxBackward0>)


In [16]:
outputs

CLIPOutput(loss=None, logits_per_image=tensor([[22.7387, 15.6618]], grad_fn=<TBackward0>), logits_per_text=tensor([[22.7387],
        [15.6618]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 0.0413, -0.0037,  0.0096,  ...,  0.0059, -0.0040,  0.0131],
        [ 0.0282, -0.0117,  0.0112,  ..., -0.0110,  0.0240,  0.0283]],
       grad_fn=<DivBackward0>), image_embeds=tensor([[ 4.4385e-02, -4.7227e-02, -1.8999e-02, -5.5320e-02, -2.4445e-02,
         -2.7468e-02,  8.0666e-03,  5.5988e-02,  6.6346e-03, -4.3237e-02,
          6.8330e-03,  5.3842e-02,  1.9102e-02, -1.9645e-02, -1.9089e-02,
         -3.6711e-02, -6.4628e-03,  3.6136e-02,  4.8417e-02, -1.2489e-02,
          1.5586e-02, -3.4931e-02, -3.0641e-02, -4.9630e-02,  3.2662e-02,
         -5.4957e-02,  3.6367e-03,  7.5559e-03,  1.0419e-02,  2.0192e-02,
         -4.6395e-03, -8.5798e-03,  4.1773e-02, -1.9621e-02, -7.7700e-02,
         -1.1550e-03,  7.6500e-02,  4.1755e-03,  1.8130e-02,  7.9161e-03,
          2.0843e-02, -6.0526e-03,  4.3

In [11]:
import torch
import torchvision

imagenet_a_directory = 'datasets/imagenet-a'
# use target transformation for label changin if needed

# Dataset: ImageNet-A
# todo do we need to pass processor as transform?
imagenet_a = torchvision.datasets.ImageFolder(root=imagenet_a_directory,transform=torchvision.transforms.ToTensor())

In [12]:
from torch.utils.data import DataLoader

# batch size
BATCH_SIZE = 64

imagenet_a_dataloader = torch.utils.data.DataLoader(imagenet_a, batch_size=BATCH_SIZE, shuffle=True)

# is it needed?

In [ ]:
import numpy as np
import torch.nn as nn


class MeanEntropy(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, outputs):
        # compute mean entropy
        # todo needs to be tested
        logits = outputs - outputs.logsumexp(dim=-1, keepdim=True)
        avg_logits = logits.logsumexp(dim=0) - np.log(logits.shape[0])
        min_real = torch.finfo(avg_logits.dtype).min
        avg_logits = torch.clamp(avg_logits, min=min_real)
        mean_entropy= -(avg_logits * torch.exp(avg_logits)).sum(dim=-1), avg_logits
        return mean_entropy


loss_fn = MeanEntropy()

In [ ]:
augmentations = []
    # # todo what are the augementation: AugMix
    # probably in utils.train_helpers.prepare_test_data

In [ ]:
def memo_tta(pretrained_model, sample, optimizer):
    augmented_samples=[]
    # todo how to parallelize this
    for augmentation in augmentations:
        augmented_sample = augmentation(sample)
        augmented_samples.append(augmented_sample)
   
    output_distributions=[]
    
    pretrained_model.eval()
    for augmented_sample in augmented_samples:
        output_distribution = pretrained_model(augmented_sample)
        output_distributions.append(output_distribution)
    
    loss = loss_fn(output_distributions)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # todo Adapting BN statistics
    return pretrained_model

In [ ]:
# todo what is the lr in paper
learning_rate = .1

def reset_model(pretrained_model): 
    pretrained_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
    optimizer = torch.optim.SGD(pretrained_model.parameters(), lr=learning_rate)
    
    return pretrained_model, optimizer

In [ ]:
# implementation of: test time adaptation via MEMO

# "only one gradient step per test point" as paper
epoch_for_adaptation = 1


for sample in imagenet_a:
    pretrained_model, optimizer = reset_model()
    adapted_model = None
    for epoch  in range(epoch_for_adaptation):
        adapted_model = memo_tta(pretrained_model, sample, optimizer)
    
    # inference phase
    with torch.no_grad():
        prediction = adapted_model(sample['source'])
        loss = loss_for_top_1_acc(prediction, sample['target'])
    # log inference accuracy